En este archivo vamos a preprocesar el texto de manera que saquemos las etiquetas de cada tipo de palabra (POS tagging). Después pasaremos texto + etiquetas a un cierto formato, que será el que le pasemos al modelo entrenado para que resuelva las correferencias en el texto. Cabe destacar que el texto que vamos a usar está en español.

### Al terminar, sacar hacer el requisitos.txt

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [2]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


# 1 Sacar etiquetas
Como hemos mencionado antes, primero vamos a sacar las etiquetas de cada palabra en un texto, comprobaremos el funcionamiento de varios modelos y discutiremos con cuál nos quedamos.

## 1.1 Textos a etiquetar
El primer paso es definir los textos de los que vamos a sacar las etiquetas. Hemos recogido 12 frases cortas para ver el funcionamiento de los modelos, creemos que es suficiente para elegir un modelo.

In [3]:
texts = [
    'El gato come pescado.',
    'El equipo de fútbol jugó su partido. Este ganó con facilidad.',
    'El coche rojo se estropeó, así que lo llevé al taller.',
    'La presidenta y el director se reunieron; ella habló primero.',
    'Entregué el informe a la jefa después de que ella lo leyera.',
    'Los equipos trabajaron duro, y al final ellos ganaron el premio.',
    'A pesar de sus problemas, el artista terminó su obra.',
    'El hermano de María dijo que él vendría.',
    'En su oficina, el abogado revisó los documentos.',
    'El libro que leí es fascinante; este autor siempre sorprende.',
    'El gato persiguió al ratón, pero este logró escapar.',
    'Hablé con Pedro sobre su proyecto y luego él me envió los archivos.',
    'Ayer hablé con Juan, le dije: "dímelo, por favor", y no me lo quiso decir',
    'Dímelo',
    'Di me lo'
]

## 1.2 Modelos
Antes de pasar a la acción vamos a definir los imports pertinentes y descargar algún modelo si fuera necesario

In [4]:
import spacy
import stanza

c:\Users\ivaga\Desktop\Proyectos programacion\Python\PLN\Desambiguador-correferencial\Entorno\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
spacy.cli.download('es_core_news_sm')
print('-'*80)
spacy.cli.download('es_dep_news_trf')
print('-'*80)
stanza.download('es')

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
--------------------------------------------------------------------------------
✔ Download and installation successful
You can now load the package via spacy.load('es_dep_news_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
--------------------------------------------------------------------------------


2025-12-10 11:41:08 INFO: Downloaded file to C:\Users\ivaga\stanza_resources\resources.json
2025-12-10 11:41:08 INFO: Downloading default packages for language: es (Spanish) ...
2025-12-10 11:42:53 INFO: Downloaded file to C:\Users\ivaga\stanza_resources\es\default.zip
2025-12-10 11:42:59 INFO: Finished downloading models and saved to C:\Users\ivaga\stanza_resources


## 1.3 Pruebas
Vamos a empezar a probar modelos sobre todas las frases que hemos definido antes.

### 1.3.1 Spacy - es_core_news_sm

In [6]:
nlp = spacy.load("es_core_news_sm")  # Para español
docs = []
for text in texts: docs.append(nlp(text))
for doc in docs:
    for token in doc:
        print(token.text, token.pos_)
    print()

El DET
gato NOUN
come VERB
pescado ADJ
. PUNCT

El DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
Este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

El DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
al ADP
taller NOUN
. PUNCT

La DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

Entregué PROPN
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

Los DET
equipos NOUN
trabajaron VERB
duro ADJ
, PUNCT
y CCONJ
al ADP
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

A ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

El DET
hermano NOUN
de ADP
María PROPN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

En ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

El DET
libro NOU

En general lo hace muy bien, aunque confunde VERB que comienzan una oración con PROPN (sustantivos propios) y, en la primera oración, confunde pescado ADJ por NOUN, por lo que podemos intuir que cometerá más veces ese error. Por lo que a nuestra futura tarea respecta, el primer error puede ser garrafal, pues podría pensar el modelo que un pronombre se refiere a un verbo de esos mal etiquetados. Además, el verbo decir más los enclíticos me y lo lo considera un sustantivo, fallo enorme ya que son dos pronombres tras el verbo.
Veamos el resto de modelos.

### 1.3.2 Spacy - es_dep_news_trf

In [7]:
nlp = spacy.load("es_dep_news_trf")
docs = []
for text in texts: docs.append(nlp(text))
for doc in docs:
    for token in doc:
        print(token.text, token.pos_)
    print()

ValueError: [E002] Can't find factory for 'curated_transformer' for language Spanish (es). This usually happens when spaCy calls `nlp.create_pipe` with a custom component name that's not registered on the current language class. If you're using a custom component, make sure you've added the decorator `@Language.component` (for function components) or `@Language.factory` (for class components).

Available factories: merge_noun_chunks, merge_entities, merge_subtokens, es.lemmatizer

En este caso vemos que corrige todos los errores previos, salvo el caso de los enclíticos, que los considera verbos (bien, pero solo la raíz es el verbo). Esto muestra que este modelo es mejor, pero no perfecto.

### 1.3.3 Stanza

In [ ]:
nlp = stanza.Pipeline('es', verbose=False)
docs = []
for text in texts: docs.append(nlp(text))
for doc in docs:
    for sentence in doc.sentences:
        for word in sentence.words:
            print(word.text, word.pos)
    print()

El DET
gato NOUN
come VERB
pescado NOUN
. PUNCT

El DET
equipo NOUN
de ADP
fútbol NOUN
jugó VERB
su DET
partido NOUN
. PUNCT
Este PRON
ganó VERB
con ADP
facilidad NOUN
. PUNCT

El DET
coche NOUN
rojo ADJ
se PRON
estropeó VERB
, PUNCT
así ADV
que SCONJ
lo PRON
llevé VERB
a ADP
el DET
taller NOUN
. PUNCT

La DET
presidenta NOUN
y CCONJ
el DET
director NOUN
se PRON
reunieron VERB
; PUNCT
ella PRON
habló VERB
primero ADV
. PUNCT

Entregué VERB
el DET
informe NOUN
a ADP
la DET
jefa NOUN
después ADV
de ADP
que SCONJ
ella PRON
lo PRON
leyera VERB
. PUNCT

Los DET
equipos NOUN
trabajaron VERB
duro ADJ
, PUNCT
y CCONJ
a ADP
el DET
final NOUN
ellos PRON
ganaron VERB
el DET
premio NOUN
. PUNCT

A ADP
pesar NOUN
de ADP
sus DET
problemas NOUN
, PUNCT
el DET
artista NOUN
terminó VERB
su DET
obra NOUN
. PUNCT

El DET
hermano NOUN
de ADP
María PROPN
dijo VERB
que SCONJ
él PRON
vendría VERB
. PUNCT

En ADP
su DET
oficina NOUN
, PUNCT
el DET
abogado NOUN
revisó VERB
los DET
documentos NOUN
. PUNCT

El D

En este caso el modelo funciona muy similar a Spacy con es_dep_news_trf. Tampoco identifica correctamente los pronombres en "dímelo", y en este caso lo marca como interjección (INTJ). Cuando solo ponemos el verbo con enclíticos (Dímelo) lo considera verbo, y cuando lo separamos con espacios las partes analiza bien los pronombres pero el verbo vuelve a confundirlo con interjección. 

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

# Cargar modelo POS (multilingual)
tagger = SequenceTagger.load("pos-multi")

sentence = Sentence("Dímelo si puedes.")

tagger.predict(sentence)

for token in sentence:
    print(token.text, token.get_labels()[0].value)


2025-12-06 15:59:47,283 SequenceTagger predicts: Dictionary with 17 tags: NOUN, PUNCT, ADP, VERB, ADJ, DET, PROPN, ADV, PRON, AUX, CCONJ, NUM, SCONJ, PART, X, SYM, INTJ
Dímelo NOUN
si PRON
puedes VERB
. PUNCT


In [ ]:
from ufal.udpipe import Model, Pipeline

model = Model.load("spanish-ancora-ud-2.1-20180111.udpipe")
pipeline = Pipeline(model, "tokenize", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu")

text = "Dímelo rápidamente."

output = pipeline.process(text)
print(output)   # formato CoNLL-U

# Parsear CoNLL-U a algo usable
for line in output.split("\n"):
    if line.startswith("#") or not line.strip():
        continue
    cols = line.split("\t")
    form, upos, lemma = cols[1], cols[3], cols[2]
    print(form, upos, lemma)


# newdoc
# newpar
# sent_id = 1
# text = Dímelo rápidamente.
1	Dímelo	Dímelo	VERB	VERB	Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin	0	root	_	_
2	rápidamente	rápidamente	ADV	ADV	_	1	advmod	_	SpaceAfter=No
3	.	.	PUNCT	PUNCT	PunctType=Peri	1	punct	_	SpacesAfter=\n


Dímelo VERB Dímelo
rápidamente ADV rápidamente
. PUNCT .


In [ ]:
# Primero prueba solo importar el módulo
from ufal.udpipe import Model

# Si funciona, prueba cargar un modelo simple
model = Model.load("spanish-ancora-ud-2.1-20180111.udpipe")
if model is None:
    print("Me cago en todo socio")
print("Modelo cargado exitosamente")

# Si falla aquí, el problema es el archivo .udpipe

Modelo cargado exitosamente


In [ ]:
from ufal.udpipe import Pipeline
pipeline = Pipeline(model, "tokenize", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu")

In [ ]:
text = "Dímelo rápidamente."

In [ ]:
output = pipeline.process(text)

In [ ]:
try:
    import requests
    print("Usando requests para descargar...")
    
    url = "https://github.com/UniversalDependencies/UD_Spanish-GSD/raw/master/spanish-gsd-ud-2.12-udpipe.udpipe"
    response = requests.get(url, verify=False)  # verify=False ignora SSL
    
    with open("spanish-gsd-ud-2.12-udpipe.udpipe", "wb") as f:
        f.write(response.content)
    
    print(f"✓ Descargado: {len(response.content)} bytes")
    
except ImportError:
    print("Instala requests: pip install requests")

Usando requests para descargar...


c:\Users\Jorge\Documents\Proyectos de programación\PLN\Desambiguador-correferencias\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'github.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✓ Descargado: 292117 bytes


In [ ]:
# Script completo que incluye modelo alternativo si falla la descarga
import os
import urllib.request
import ssl
from ufal.udpipe import Model, Pipeline

def download_model():
    """Descarga el modelo con múltiples intentos"""
    model_path = "spanish_model.udpipe"
    
    if os.path.exists(model_path) and os.path.getsize(model_path) > 10000:
        print(f"✓ Modelo ya existe: {os.path.getsize(model_path)} bytes")
        return model_path
    
    print("Descargando modelo UDPipe para español...")
    
    # URL alternativa más confiable
    url = "https://raw.githubusercontent.com/UniversalDependencies/UD_Spanish-GSD/master/spanish-gsd-ud-2.12-udpipe.udpipe"
    
    try:
        # Configurar contexto SSL más permisivo
        ctx = ssl.create_default_context()
        ctx.check_hostname = False
        ctx.verify_mode = ssl.CERT_NONE
        
        # Descargar
        req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        with urllib.request.urlopen(req, context=ctx) as response:
            data = response.read()
            
        with open(model_path, 'wb') as f:
            f.write(data)
            
        print(f"✓ Modelo descargado: {len(data)} bytes")
        return model_path
        
    except Exception as e:
        print(f"✗ Error descargando: {e}")
        
        # Crear un modelo mínimo de emergencia
        print("Creando modelo de emergencia...")
        create_dummy_model(model_path)
        return model_path

def create_dummy_model(path):
    """Crea un archivo .udpipe básico para pruebas"""
    # Esto es solo para que UDPipe no falle
    dummy_content = b"UDPIPE MODEL v1\n\n\n"
    with open(path, 'wb') as f:
        f.write(dummy_content)
    print("✓ Modelo dummy creado para pruebas")

# Descargar o usar modelo existente
model_path = download_model()

# Intentar cargar el modelo
print(f"\nCargando modelo desde: {model_path}")
model = Model.load(model_path)

if model is None:
    print("✗ ERROR: No se pudo cargar el modelo.")
    print("Posibles causas:")
    print("1. Archivo corrupto")
    print("2. Versión incompatible de ufal.udpipe")
    print("3. El archivo no es realmente un modelo UDPipe")
    
    # Verificar el archivo
    if os.path.exists(model_path):
        with open(model_path, 'rb') as f:
            first_bytes = f.read(100)
            print(f"Primeros bytes del archivo: {first_bytes[:50]}")
else:
    print("✓ Modelo cargado exitosamente!")
    
    # Probar con texto simple
    pipeline = Pipeline(model, "tokenize", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu")
    text = "Hola mundo."
    
    try:
        output = pipeline.process(text)
        print("✓ Pipeline funcionando!")
        print("\nResultado:")
        for line in output.split("\n"):
            if line and not line.startswith("#"):
                print(line)
    except Exception as e:
        print(f"✗ Error en pipeline: {e}")

Descargando modelo UDPipe para español...
✗ Error descargando: HTTP Error 404: Not Found
Creando modelo de emergencia...
✓ Modelo dummy creado para pruebas

Cargando modelo desde: spanish_model.udpipe
✗ ERROR: No se pudo cargar el modelo.
Posibles causas:
1. Archivo corrupto
2. Versión incompatible de ufal.udpipe
3. El archivo no es realmente un modelo UDPipe
Primeros bytes del archivo: b'UDPIPE MODEL v1\n\n\n'
